# 🔐 AES + LSB Steganografi – Mesaj Gizleme

In [1]:
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def aes_sifrele(mesaj):
    key = get_random_bytes(32)  # AES-256 anahtarı
    iv = get_random_bytes(12)   # 12 byte nonce (GCM için)
    cipher = AES.new(key, AES.MODE_GCM, nonce=iv)
    ciphertext, tag = cipher.encrypt_and_digest(mesaj.encode('utf-8'))
    sifreli_veri = iv + tag + ciphertext
    return key, sifreli_veri

def aes_gom(gorsel_yolu, veri, kayit_adi="C_encoded_aes.png"):
    binary_veri = ''.join(format(b, '08b') for b in veri) + '1111111111111111'
    img = Image.open(gorsel_yolu)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    encoded = img.copy()
    data_idx = 0
    for y in range(encoded.size[1]):
        for x in range(encoded.size[0]):
            r, g, b = encoded.getpixel((x, y))
            rgb = [r, g, b]
            for i in range(3):
                if data_idx < len(binary_veri):
                    rgb[i] = (rgb[i] & ~1) | int(binary_veri[data_idx])
                    data_idx += 1
            encoded.putpixel((x, y), tuple(rgb))
            if data_idx >= len(binary_veri):
                encoded.save(kayit_adi)
                print(f"[✓] Mesaj başarıyla AES ile şifrelenip gömüldü: {kayit_adi}")
                return

In [ ]:
# Örnek kullanım
mesaj = "Bybit saldırısı başlatıldı."
key, sifreli_veri = aes_sifrele(mesaj)
aes_gom("C.png", sifreli_veri)
print("AES Anahtar (base64):", key.hex())

# 🔓 AES + LSB Steganografi – Mesaj Çözme

In [3]:
from PIL import Image
from Crypto.Cipher import AES
import binascii

def aes_coz(gorsel_yolu, key_hex):
    key = bytes.fromhex(key_hex)
    img = Image.open(gorsel_yolu)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    binary_data = ""
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            for color in img.getpixel((x, y)):
                binary_data += str(color & 1)
    byte_data = []
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        if byte == '11111111' and binary_data[i+8:i+16] == '11111111':
            break
        byte_data.append(int(byte, 2))
    veri = bytes(byte_data)
    iv, tag, ciphertext = veri[:12], veri[12:28], veri[28:]
    cipher = AES.new(key, AES.MODE_GCM, nonce=iv)
    try:
        return cipher.decrypt_and_verify(ciphertext, tag).decode('utf-8')
    except Exception:
        return "[❌] Anahtar hatalı veya veri bozulmuş."


In [ ]:
aes_coz("C_encoded_aes.png", "7b534294fe37841d1bc2018f8372139808c5f99e1ecc79a1132c3e6bc1b12c62")

In [ ]:
# Örnek kullanım (AES anahtarı hex formatında girilmeli)
# aes_coz("A_encoded_aes.png", "buraya_hex_anahtar")